In [1]:
import gc
gc.collect()

200

In [2]:
# 各ライブラリのインストール
# Readme参照

In [3]:
!nvidia-smi

Fri Jul  4 03:45:06 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    On   | 00000000:19:00.0 Off |                  Off |
| 61%   68C    P8    20W / 140W |    163MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A4000    On   | 00000000:1A:00.0 Off |                  Off |
| 65%   

In [4]:
!pip show torch torchvision

Name: torch
Version: 2.7.1+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu11, nvidia-cuda-cupti-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, nvidia-cufft-cu11, nvidia-curand-cu11, nvidia-cusolver-cu11, nvidia-cusparse-cu11, nvidia-nccl-cu11, nvidia-nvtx-cu11, setuptools, sympy, triton, typing-extensions
Required-by: pytorch-lightning, torchaudio, torchmetrics, torchvision
---
Name: torchvision
Version: 0.22.1+cu118
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/pyth

# CONFIG

In [5]:
import os

class CONFIG:
    
    # デバックモード
    is_debug_mode = False
    max_class_num = 5

    # 学習対象のファイル形式
    use_mxrecord = False
    # varidationファイルを作成するか
    make_validation_memfiles = False

    # データセットの指定(オリジナルデータセットかそうでないか)
    is_original_dataset = True
    ### オリジナルデータセットのコピー元：ImageFolder形式に変換する前
    original_src_root_path = "/data1/share/Datasets/Face/VGGFace2_frontal"
    ### 訓練データ
    dataset_name = 'faces_webface_112x112' if not is_original_dataset else 'original_train_dataset'
    dataset_dir = os.path.join("data", dataset_name)
    ### テストデータ
    test_dataset_name = 'faces_webface_112x112' if not is_original_dataset else 'original_test_dataset'
    test_dataset_dir = os.path.join("data", test_dataset_name)

In [6]:
import gdown
import zipfile
import pandas as pd
from convert_train_dataset import convert_to_imagefolder

if not os.path.isdir(CONFIG.dataset_dir):
    if CONFIG.dataset_name == 'faces_umd':
        #UMD-facesをダウンロード
        file_id = '1azhEHoJjVmifuzBVKJwl-sDbLZ-Wzp4O'
        gdown.download(f"https://drive.google.com/uc?id={file_id}", f"{CONFIG.dataset_name}.zip", quiet=False)
        #zip解凍
        with zipfile.ZipFile(f"{CONFIG.dataset_name}.zip", 'r') as zip_ref:
            zip_ref.extractall("data")  # 任意の展開先フォルダ
    if CONFIG.dataset_name == 'faces_webface_112x112':
        # CASIA-webfaceをダウンロード
        file_id = '1KxNCrXzln0lal3N4JiYl9cFOIhT78y1l'
        gdown.download(f"https://drive.google.com/uc?id={file_id}", f"{CONFIG.dataset_name}.zip", quiet=False)
        #zip解凍
        with zipfile.ZipFile(f"{CONFIG.dataset_name}.zip", 'r') as zip_ref:
            zip_ref.extractall("data")  # 任意の展開先フォルダ

if CONFIG.is_original_dataset and not os.path.isdir(CONFIG.dataset_dir) and not os.path.isdir(CONFIG.test_dataset_dir):
    #訓練用、検証用csvファイルを読み込み、重複を削除、test_recognizer_train_list.csvを上書き保存
    train_csv_path = 'data/target_recognizer_train_list.csv'
    test_csv_path = 'data/eval_recognizer_train_list.csv'
    test_df = pd.read_csv(test_csv_path)
    train_df = pd.read_csv(train_csv_path)
    train_paths = set(train_df['File Path'])
    initial_len = len(test_df)

    # testデータから train と重複している行を除外
    filtered_test_df = test_df[~test_df['File Path'].isin(train_paths)]

    print(f"Removed {initial_len - len(filtered_test_df)} duplicate entries from test.")

    # 上書き保存
    filtered_test_df.to_csv(test_csv_path, index=False)
    print(f"Updated {test_csv_path} successfully.")

    # オリジナルデータセットのディレクトリを作成
    os.makedirs(CONFIG.dataset_dir, exist_ok=True)
    os.makedirs(CONFIG.test_dataset_dir, exist_ok=True)

    # データセットをImageFolder形式でコピー
    convert_to_imagefolder(train_csv_path,CONFIG.original_src_root_path, os.path.join(CONFIG.dataset_dir,"imgs"))
    convert_to_imagefolder(test_csv_path,CONFIG.original_src_root_path , os.path.join(CONFIG.test_dataset_dir,"imgs"))

# rec file convert to image files

In [7]:
# recordIO形式をImageFile形式に変換する（dataset_dir直下にimgsディレクトリが生成される）
if not os.path.exists(f"{CONFIG.dataset_dir}/imgs") and not CONFIG.is_original_dataset and not CONFIG.use_mxrecord: # InsightFaceDataset
    !python AdaFace/convert.py --rec_path {CONFIG.dataset_dir} --make_image_files 
elif not os.path.exists(f"{CONFIG.dataset_dir}/train.rec") and CONFIG.is_original_dataset and not CONFIG.use_mxrecord : # OriginalFaceDataset
    pass
else:
    pass

In [8]:
data_root = os.path.dirname(CONFIG.dataset_dir) 
dataset_dir = CONFIG.dataset_dir
dataset_img_dir = os.path.join(CONFIG.dataset_dir, "imgs")
test_dataset_dir = CONFIG.test_dataset_dir
test_dataset_img_dir = os.path.join(CONFIG.test_dataset_dir, "imgs")

print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}") 
print(f" data_root: {data_root}\n test_dataset_dir: {test_dataset_dir}\n test_dataset_img_dir:{test_dataset_img_dir}")

# データの内容をサマライズ
from summarize_ImageFolder import summarize_imagefolder
summarize_imagefolder(dataset_img_dir)
summarize_imagefolder(test_dataset_img_dir)

 data_root: data
 dataset_dir: data/original_train_dataset
 dataset_img_dir:data/original_train_dataset/imgs
 data_root: data
 test_dataset_dir: data/original_test_dataset
 test_dataset_img_dir:data/original_test_dataset/imgs


KeyboardInterrupt: 

In [ ]:
import shutil

debug_dataset_dir = os.path.join(data_root, "debug_dataset")
debug_img_dir = os.path.join(debug_dataset_dir, "imgs") 

# デバッグモードの場合、max_class_num分dataset_dirのimgsファイルをコピー、対象データディレクトリをdebug_datasetディレクトリに変更
if CONFIG.is_debug_mode:

    # 既存フォルダをクリア
    if os.path.isdir(debug_dataset_dir):
        os.remove(f"{debug_dataset_dir}/train.idx") if os.path.exists(f"{debug_dataset_dir}/train.idx") else None
        os.remove(f"{debug_dataset_dir}/train.lst") if os.path.exists(f"{debug_dataset_dir}/train.lst") else None
        os.remove(f"{debug_dataset_dir}/train.rec") if os.path.exists(f"{debug_dataset_dir}/train.rec") else None
        shutil.rmtree(f"{debug_dataset_dir}/imgs") 
    os.makedirs(debug_dataset_dir, exist_ok=True)

    # クラス名ディレクトリをソートして先頭5つ取得
    class_dirs = sorted([
        d for d in os.listdir(dataset_img_dir)
        if os.path.isdir(os.path.join(dataset_img_dir, d))
    ])[:CONFIG.max_class_num]

    # 画像データをImageFolder形式としてコピー
    for cls in class_dirs:
        src = os.path.join(dataset_img_dir, cls)
        dst = os.path.join(debug_img_dir, cls)
        shutil.copytree(src, dst)
        # もしくは高速に済ませたいなら symlink:
        # os.symlink(src, dst, target_is_directory=True)

    # dataset_dirをdebug_dataset_dirに変更
    dataset_dir = debug_dataset_dir
    dataset_img_dir = debug_img_dir

    print("============== debug file is created ============")
    
else:
    print("==== you're using raw data (not using debug directory)")

print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}")

# recordIO形式で訓練する場合、.recに変換
if CONFIG.use_mxrecord:
    print("========== create new rec file =========")
    !python create_train_rec.py --img_dir {dataset_img_dir} --output_dir {dataset_dir}
    print(".rec .lst .bin is created")

dataset_dir_name = os.path.basename(dataset_dir)
dataset_img_dir_name = os.path.basename(dataset_img_dir)
print(f" data_root: {data_root}\n dataset_dir: {dataset_dir}\n dataset_img_dir:{dataset_img_dir}")
print(f"▶ Using dataset_path : {dataset_dir}")

============== debug file is created ============
 data_root: data
 dataset_dir: data/debug_dataset
 dataset_img_dir:data/debug_dataset/imgs
 data_root: data
 dataset_dir: data/debug_dataset
 dataset_img_dir:data/debug_dataset/imgs
▶ Using dataset_path : data/debug_dataset


In [ ]:
import os

# CASIA webfaceデータセットから対象.binファイルを事前にコピー済み
validation_dir = os.path.join(data_root, "validation_dataset")
val_dataset_dirs = ["agedb_30", "calfw", "cfp_ff", "cfp_fp", "cplfw", "lfw"]
val_dataset_dir_name = os.path.basename(validation_dir)

if all(os.path.exists(f"{validation_dir}/{d}") for d in val_dataset_dirs):
    print("val dir is already exists")
else:
    print("some val data is not exists : convert .bin to new val data")
    # 検証ファイルを構築
    !python AdaFace/convert.py --rec_path {validation_dir} --make_validation_memfiles

val dir is already exists


# train

In [ ]:
# !python AdaFace/main.py --help

訓練データに使用するクラス数とクラスごとのデータ数を変えて分散の出力に影響が出るか実験

In [ ]:
# # !NUM_CLASSES=$(find "${DATA_ROOT}/${DATASET_NAME}/imgs" -mindepth 1 -maxdepth 1 -type d | wc -l)
# print(f"data_root:{data_root}, dataset_dir_name:{dataset_dir_name} ")
# # 使用中の場合は、gpuのnoを指定：CUDA_VISIBLE_DEVICES=2
# !python AdaFace/main.py \
#     --data_root {data_root} \
#     --train_data_path {dataset_dir_name} \
#     --val_data_path {val_dataset_dir_name} \
#     --prefix ir101_ms1mv2_adaface \
#     --gpus 2 \
#     --use_16bit \
#     --arch ir_101 \
#     --batch_size 64 \
#     --num_workers 16 \
#     --epochs 5 \
#     --lr_milestones 12,20,24 \
#     --lr 0.1 \
#     --head adaface \
#     --m 0.4 \
#     --h 0.333 \
#     --low_res_augmentation_prob 0.2 \
#     --crop_augmentation_prob 0.2 \
#     --photometric_augmentation_prob 0.2 

In [ ]:
# ckpt_path = "experiments/ir101_ms1mv2_adaface_07-03_1/last.ckpt"
# !python extract_and_analyze.py \
#   --ckpt_path {ckpt_path}\
#   --img_dir {test_dataset_img_dir} \
#   --batch_size 64 \
#   --num_workers 4 \
#   --device cuda


In [ ]:
import os
import shutil
import random
from pathlib import Path
from PIL import Image
import subprocess
import time

# ==== パラメータ設定 ====
data_root = Path("data")
original_dataset_path = data_root / dataset_dir_name 
original_dataset_imgs_path = original_dataset_path / "imgs"
val_data_path = "validation_dataset"
experiment_prefix = "exp_variance_test"
tmp_dataset_path = data_root / "tmp_dataset"
extract_script_path = "extract_and_analyze.py"
gpu_num = 2

experiment_configs = [
    {"class_ratio": 1.0, "sample_ratio": 1.0},
    {"class_ratio": 0.5, "sample_ratio": 1.0},
    {"class_ratio": 1.0, "sample_ratio": 0.5},
    {"class_ratio": 0.5, "sample_ratio": 0.5},
]

# ==== 実験のループ開始 ====
for idx, config in enumerate(experiment_configs):
    class_ratio = config["class_ratio"]
    sample_ratio = config["sample_ratio"]
    print(f"\n=== Experiment {idx+1}: class_ratio={class_ratio}, sample_ratio={sample_ratio} ===")

    # 1. tmp_datasetの作成（存在する場合は削除）
    if tmp_dataset_path.exists():
        shutil.rmtree(tmp_dataset_path)
    tmp_imgs_path = tmp_dataset_path / "imgs"
    tmp_imgs_path.mkdir(parents=True, exist_ok=True)

    # 2. クラスディレクトリの取得と選択
    class_dirs = sorted([d for d in original_dataset_imgs_path.iterdir() if d.is_dir()])
    selected_class_dirs = random.sample(class_dirs, int(len(class_dirs) * class_ratio))

    for class_dir in selected_class_dirs:
        images = sorted(class_dir.glob("*.png"))
        selected_images = random.sample(images, max(1, int(len(images) * sample_ratio)))
        dest_dir = tmp_imgs_path / class_dir.name
        dest_dir.mkdir(parents=True, exist_ok=True)
        for img_path in selected_images:
            shutil.copy2(img_path, dest_dir / img_path.name)

    print(f"[INFO] Copied {len(selected_class_dirs)} classes to {tmp_imgs_path}")

    # 3. main.py 実行
    cmd = [
        "python", "AdaFace/main.py",
        "--data_root", str(data_root),
        "--train_data_path", tmp_dataset_path.name,
        "--val_data_path", val_data_path,
        "--prefix", f"{experiment_prefix}_{idx+1}",
        "--gpus", str(gpu_num),
        "--use_16bit",
        "--arch", "ir_101",
        "--batch_size", "64",
        "--num_workers", "4",
        "--epochs", "6",
        "--lr_milestones", "12,20,24",
        "--lr", "0.1",
        "--head", "adaface",
        "--m", "0.4",
        "--h", "0.333",
        "--low_res_augmentation_prob", "0.2",
        "--crop_augmentation_prob", "0.2",
        "--photometric_augmentation_prob", "0.2"
    ]
    subprocess.run(cmd, check=True)

    # 4. 最新 experiments ディレクトリの取得
    exp_dir = Path("experiments")
    latest_exp_dir = max(exp_dir.glob("*"), key=os.path.getmtime)
    ckpt_path = latest_exp_dir / "last.ckpt"
    print(f"[INFO] Latest experiment directory: {latest_exp_dir}")

    # 5. extract_and_analyze.py を train データに対し実行
    extract2train_cmd = [
        "python", extract_script_path,
        "--ckpt_path", str(ckpt_path),
        "--img_dir", str(tmp_imgs_path),
        "--batch_size", "64",
        "--num_workers", "4"
    ]
    subprocess.run(extract2train_cmd, check=True)
    print(f"[INFO] Ran extract_and_analyze.py to training data for experiment  {idx+1}")

    # 6. extract_and_analyze.py を testデータに対し実行
    extract2test_cmd = [
        "python", extract_script_path,
        "--ckpt_path", str(ckpt_path),
        "--img_dir", test_dataset_img_dir,
        "--batch_size", "64",
        "--num_workers", "4"
    ]
    subprocess.run(extract2test_cmd, check=True)

    print(f"[INFO] Ran extract_and_analyze.py to training data for experiment  {idx+1}")
    # 7. 後始末：experimentログとtmpデータ削除
    shutil.rmtree(latest_exp_dir)
    shutil.rmtree(tmp_dataset_path)
    print(f"[INFO] Cleaned up experiment {idx+1}")




=== Experiment 1: class_ratio=1.0, sample_ratio=1.0 ===
[INFO] Copied 5 classes to data/tmp_dataset/imgs
パースされた epochs: 1
classnum: 5

\AdaFace with the following property
self.m 0.4
self.h 0.333
self.s 64.0
self.t_alpha 0.01
start training


Global seed set to 42
/data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=2)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=2)` instead.
  rank_zero_deprecation(
/data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/lightning_lite/plugins/environments/slurm.py:167: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python AdaFace/main.py --data_root data --train_data_path t ...
  rank_zero_warn(
Using 16bit native Automatic Mixed Precision (AMP)
/data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/pytorch_lightning/plugins/precision/native_amp.py:56: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated

パースされた epochs: 1
classnum: 5

\AdaFace with the following property
self.m 0.4
self.h 0.333
self.s 64.0
self.t_alpha 0.01
start training


[rank: 1] Global seed set to 42
/data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/pytorch_lightning/plugins/precision/native_amp.py:56: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
[rank: 1] Global seed set to 42
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------



creating train dataset
creating train dataset
creating val dataset
laoding validation data memfile
laoding validation data memfile
creating val dataset
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
Sanity Checking: 0it [00:00, ?it/s]

...

Epoch 0:   0%|          | 1/486 [00:00<04:30,  1.79it/s, loss=25.4, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 0:  76%|███████▋  | 371/486 [00:55<00:17,  6.67it/s, loss=25.4, v_num=0]start evaluating
evaluating from  


/data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


start evaluating
evaluating from  


KeyboardInterrupt: 

[rank1]:[W704 03:44:39.697989590 TCPStore.cpp:125] [c10d] recvValue failed on SocketImpl(fd=18, addr=[::ffff:127.0.0.1]:59294, remote=[::ffff:127.0.0.1]:47657): failed to recv, got 0 bytes
Exception raised from recvBytes at /pytorch/torch/csrc/distributed/c10d/Utils.hpp:678 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x98 (0x14643ee955e8 in /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x5ba8b0e (0x146483b92b0e in /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/torch/lib/libtorch_cpu.so)
frame #2: <unknown function> + 0x5baae50 (0x146483b94e50 in /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/site-packages/torch/lib/libtorch_cpu.so)
frame #3: <unknown function> + 0x5bab75a (0x146483b9575a in /data2/tomoshi0514/anaconda3/envs/AdafaceTest/lib/python3.12/si